## [--- Day 9: Movie Theater ---](https://adventofcode.com/2025/day/9)

In [ ]:
from typing import ClassVar


class vector2(tuple[int, int]):
    __slots__ = ()  # Prevent '__dict__' creation

    """Crude 2D vector wrapper for tuples"""

    def __new__(cls, x: int = 0, y: int = 0) -> "vector2":
        return super().__new__(cls, (x, y))

    zero: ClassVar["vector2"]
    up: ClassVar["vector2"]
    down: ClassVar["vector2"]
    left: ClassVar["vector2"]
    right: ClassVar["vector2"]

    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]

    def rotated_left(self) -> "vector2":
        return vector2(self[1], -self[0])

    def rotated_right(self) -> "vector2":
        return vector2(-self[1], self[0])

    def reflected_around(self, origin: "vector2") -> "vector2":
        return vector2(-self[0] + 2 * origin[0], -self[1] + 2 * origin[1])

    def distance_to(self, other: "vector2") -> int:
        return abs(self[0] - other[0]) + abs(self[1] - other[1])

    def rect_area(self, other: "vector2") -> int:
        return (abs(self[0] - other[0]) + 1) * (abs(self[1] - other[1]) + 1)

    def __add__(self, other: "vector2") -> "vector2":  # type: ignore[override]
        return vector2(self[0] + other[0], self[1] + other[1])

    def __sub__(self, other: "vector2" | tuple[int, int]) -> "vector2":
        return vector2(self[0] - other[0], self[1] - other[1])

    def __repr__(self) -> str:
        return f"({self[0]}, {self[1]})"


vector2.zero = vector2(0, 0)
vector2.up = vector2(0, -1)
vector2.down = vector2(0, 1)
vector2.left = vector2(-1, 0)
vector2.right = vector2(1, 0)


from math import inf


class Grid:
    def __init__(self, data: list[vector2]) -> None:
        self.data = data

        xs = [p.x for p in self.data]
        ys = [p.y for p in self.data]
        self.min_x, self.max_x = min(xs), max(xs)
        self.min_y, self.max_y = min(ys), max(ys)

        self.width = self.max_x - self.min_x + 1
        self.height = self.max_y - self.min_y + 1

        # base: list[list[str]] = [['.'] * self.width for _ in range(self.height)]

        # for pos in data:
        #     base[pos.y - self.min_y][pos.x - self.min_x] = '#'

        # self.base = base

    def in_bounds(self, pos: vector2) -> bool:
        return self.min_x <= pos.x <= self.max_x and self.min_y <= pos.y <= self.max_y

    def rect_area(self, a: vector2, b: vector2) -> int:
        return (abs(a.x - b.x) + 1) * (abs(a.y - b.y) + 1)

    # def draw_rect(self, a: vector2, b: vector2, char: str = 'O') -> None:
    #     grid = [row.copy() for row in self.base]

    #     for y in range(min(a.y, b.y), max(a.y, b.y) + 1):
    #         for x in range(min(a.x, b.x), max(a.x, b.x) + 1):
    #             grid[y - self.min_y][x - self.min_x] = char

    #     print('\n'.join(["".join(row) for row in grid]))

    # def __getitem__(self, pos: vector2|tuple[int, int]) -> str:
    #     if self.in_bounds(pos):
    #         return self.base[pos[1] - self.min_y][pos[0] - self.min_x]

    # def __setitem__(self, pos: vector2|tuple[int, int], value: str) -> None:
    #     if self.in_bounds(pos):
    #         self.base[pos[1] - self.min_y][pos[0] - self.min_x] = value

    # def __repr__(self) -> str:
    #     return "\n".join("".join(row) for row in self.base)

In [ ]:
def solve() -> None:
    with open("..\\data\\09.txt") as file:
        coords = [line.strip().split(",") for line in file.readlines()]

    grid = Grid([vector2(int(x), int(y)) for x, y in coords])

    # Horizons
    north_east: list[vector2] = []
    south_east: list[vector2] = []
    south_west: list[vector2] = []
    north_west: list[vector2] = []

    # Lexicographical ascending sort, x then y
    grid.data.sort()
    max_y: int = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_east.append(pos)
            max_y = pos.y

    min_y: int = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_west.append(pos)
            min_y = pos.y

    # Lexicographical sort, x descending then y ascending
    grid.data.sort(key=lambda pos: (-pos.x, pos.y))
    max_y = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_west.append(pos)
            max_y = pos.y

    min_y = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_east.append(pos)
            min_y = pos.y

    non_dominated_points = set(north_east + south_east + south_west + north_west)
    dominated_points = set(grid.data) - non_dominated_points

    # for point in dominated_points:
    #     grid[point] = 'x'

    print(f"{len(dominated_points)} dominated points removed")
    # print(grid)
    print(f"{len(non_dominated_points)} non-dominated points remain\n")

    ne_sw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    ne_sw_area: int = 0

    for ne in north_east:
        for sw in south_west:
            area = ne.rect_area(sw)
            if area > ne_sw_area:
                ne_sw_area = area
                ne_sw_pair = (ne, sw)

    print(f"NE-SW Max Area: {ne_sw_area} between {ne_sw_pair[0]} and {ne_sw_pair[1]}")
    # grid.draw_rect(ne_sw_pair[0], ne_sw_pair[1])

    se_nw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    se_nw_area: int = 0

    for se in south_east:
        for nw in north_west:
            area = se.rect_area(nw)
            if area > se_nw_area:
                se_nw_area = area
                se_nw_pair = (se, nw)

    print(f"SE-NW Max Area: {se_nw_area} between {se_nw_pair[0]} and {se_nw_pair[1]}")
    # grid.draw_rect(se_nw_pair[0], se_nw_pair[1])

    print(f"The largest area between any two points is {max(ne_sw_area, se_nw_area)}")


solve()

280 dominated points removed
216 non-dominated points remain

NE-SW Max Area: 4678664144 between (16361, 84880) and (83532, 15229)
SE-NW Max Area: 4761736832 between (17804, 13266) and (86507, 82573)
The largest area between any two points is 4761736832


In [ ]:
# TODO: convert the grid into an image so it's more efficient to visualize large grids.